In [1]:
import tensorwrap as tf
tf.__version__, tf.config.set_visible_devices("cuda"), tf.config.list_physical_devices("GPU")

('0.0.0.4',
 None,
 [StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)])

In [2]:
from tensorwrap import nn

In [3]:
layer = tf.nn.layers.Dense(units = 10)
layer1 = tf.nn.layers.Dense(units = 1)

In [4]:
x = tf.range(1, 1e7, dtype=tf.float32)
x = tf.expand_dims(x, axis = 1)
y = x + 10
x /= 1e5

In [5]:
model = nn.Sequential([
    layer1
])

In [6]:
class Mo(nn.Model):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.linear = layer1
        # self.linear1 = layer
    def call(self, x):
        return self.linear(x)

In [7]:
model = Mo()

In [8]:
model.compile(
    loss = tf.nn.losses.mse,
    optimizer = tf.nn.optimizers.gradient_descent(1e-5),
)

In [9]:
%timeit model.fit(x, y, epochs = int(100))

Epoch 1|100 
[=========================]    Loss: 33333352333312.00000     Metric: 33333352333312.00000
Epoch 2|100 
[=========================]    Loss: 33265721278464.00000     Metric: 33265721278464.00000
Epoch 3|100 
[=========================]    Loss: 33198226538496.00000     Metric: 33198226538496.00000
Epoch 4|100 
[=========================]    Loss: 33130868113408.00000     Metric: 33130868113408.00000
Epoch 5|100 
[=========================]    Loss: 33063648100352.00000     Metric: 33063648100352.00000
Epoch 6|100 
[=========================]    Loss: 32996564402176.00000     Metric: 32996564402176.00000
Epoch 7|100 
[=========================]    Loss: 32929617018880.00000     Metric: 32929617018880.00000
Epoch 8|100 
[=========================]    Loss: 32862803853312.00000     Metric: 32862803853312.00000
Epoch 9|100 
[=========================]    Loss: 32796129099776.00000     Metric: 32796129099776.00000
Epoch 10|100 
[=========================]    Loss: 3272958856396

In [10]:
vars(model)

{'args': (),
 'kwargs': {},
 'trainable_variables': [],
 'layers': [<tensorwrap.nn.layers.Dense at 0x7f9c196a5d50>],
 'linear': <tensorwrap.nn.layers.Dense at 0x7f9c196a5d50>,
 'loss_fn': <CompiledFunction of <function mse at 0x7f9b56689120>>,
 'optimizer': <tensorwrap.nn.optimizers.gradient_descent at 0x7f9c196a7580>,
 'metrics': <CompiledFunction of <function mse at 0x7f9b56689120>>}